In [1]:
!pip install transformers datasets torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset


In [2]:
from transformers import pipeline
model_name = "papluca/xlm-roberta-base-language-detection"

In [3]:
pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", max_length=512, truncation=True,device=0, batch_size=500)


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
data = load_dataset("mwarchalowski/grants", split='train')

Generating train split:   0%|          | 0/143713 [00:00<?, ? examples/s]

In [5]:
res = pipe(data['abstr'])

In [6]:
clean_data = data.map(lambda exmple, idx: res[idx], with_indices=True).filter(lambda example: example['label'] == 'en').remove_columns(["label","score"])

Map:   0%|          | 0/143713 [00:00<?, ? examples/s]

Filter:   0%|          | 0/143713 [00:00<?, ? examples/s]

In [7]:
clean_data

Dataset({
    features: ['id', 'title', 'abstr'],
    num_rows: 140286
})

In [8]:
clean_data.save_to_disk("clean-data")

Saving the dataset (0/1 shards):   0%|          | 0/140286 [00:00<?, ? examples/s]

In [9]:
clean_data.push_to_hub("mwarchalowski/grants")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/141 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/361 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mwarchalowski/grants/commit/f34a3eab6d163ca0473845fffd89bbb8acf5e8a1', commit_message='Upload dataset', commit_description='', oid='f34a3eab6d163ca0473845fffd89bbb8acf5e8a1', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
def text_size(str):
    return len(str.split(" "))

In [36]:
example_lenghts = clean_data.map(lambda example: {'text_len':text_size(" ".join([example["title"], example["abstr"]]))})

In [44]:
example_lenghts.filter(lambda example: example['text_len'] < 10)[:1]

{'id': ['AKA:129695'],
 'title': ['Inorganic Materials Chemistry Graduate Program'],
 'abstr': ['Inorganic materials graduate program'],
 'text_len': [9]}

In [63]:
pd_examples = example_lenghts.with_format('pandas')[:]

In [88]:
pd_examples["abstr"].value_counts()

abstr
Public/State Controlled Institution of Higher Education                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [95]:
dups = set([i for i, v in pd_examples["abstr"].value_counts().items() if v > 10])

In [98]:
dups

{'Abstract Not Available',
 'Abstract not available',
 'Awaiting Public Project Summary',
 'Doctoral Training Partnerships: a range of postgraduate training is funded by the Research Councils. For information on current funding routes, see the common terminology at https://www.ukri.org/apply-for-funding/how-we-fund-studentships/. Training grants may be to one organisation or to a consortia of research organisations. This portal will show the lead organisation only.',
 'For-Profit Organization (Other than Small Business)',
 'Hispanic-serving Institution',
 'Historically Black Colleges and Universities (HBCUs)',
 'In the future, an increased percentage of steel will originate from the Electric Arc Furnace and from the scrap route, for both long products as well as a proportion of flat products. Since a deterioration in scrap quality is also predicted, the European steel industry could be faced with quality problems due to the higher content of tramp elements in steel, which may affect en

In [99]:
no_dups = clean_data.filter(lambda example: example["abstr"] in dups)

Filter:   0%|          | 0/140286 [00:00<?, ? examples/s]

In [100]:
no_dups

Dataset({
    features: ['id', 'title', 'abstr'],
    num_rows: 5587
})

In [101]:
no_dups[:10]

{'id': ['CDMRP:03336004',
  'CDMRP:04100002',
  'CDMRP:05187006',
  'CDMRP:05263001',
  'CDMRP:06059005',
  'CDMRP:06144019',
  'CDMRP:07040004',
  'CDMRP:07078003',
  'CDMRP:07115001',
  'CDMRP:07142001'],
 'title': ['Developing a Brief Method for the Simultaneous Assessment of Anaerobic and Aerobic Fitness',
  'The Dean and Betty Gallo Prostate Cancer Center',
  'Development of a Robust Phospho-Protein Interaction Map Useful for Diagnostic of Neuro-degenerative Diseases',
  'Integrated Neurocognitive Assessment System for Critical Military Health Needs',
  'North American Clinical Trials Network / Building Infrastructure to Accelerate Transfer of Basic Research in Spinal Cord Injury (SCI) to Clinical Practice',
  'Environmental neurotoxin exposure-induced nuclear-mitochondrial cascade mechanisms of neurodegeneration',
  'Bone Repair and Military Readiness',
  'National Oncogenomic and Molecular Imaging Center',
  'Standardizing Prosthetic Socket Design',
  'Development of Technology 